In [2]:
from googleapiclient.discovery import build
from google.oauth2 import service_account
import os
import logging
import numpy as np
import pandas as pd
from datetime import date
from dateutil.relativedelta import *


scopes = ['https://www.googleapis.com/auth/analytics.edit']
key_path = 'C:\ecommerce\keys\ga_service_account.json'
credentials = service_account.Credentials.from_service_account_file(key_path, scopes=scopes)
service = build('analytics', 'v3', credentials=credentials)

### Get all ua profiles

In [3]:
def read_profiles_per_property(service, property):
    """
    Read ua_profiles from property and format into DataFrame
    """
    property_name = property['name']                   # property name
    pid = property['id']                               # property id

    if property_name == 'Roll-Up Property':
        property_source = 'N/A'                        # get property source (app or web) and type (brand or market)
        property_type = 'N/A'
    else:
        property_source = property_name.split(' ')[2]  # property source (app or web)
        property_type = property_name.split(' ')[4]    # property type (brand or market)

    # get all profiles in this property
    profiles = service.management().profiles().list(accountId='66188758', webPropertyId=pid).execute()

    ua_profile_id = [prf['id'] for prf in profiles.get('items')]  # list of all profile id
    ua_profile_name = [prf['name'] for prf in profiles.get('items')]  # list of all profile name
    
    # put into DataFrame
    df = pd.DataFrame({'ua_profile_id': ua_profile_id,
                       'name': ua_profile_name,
                       'property_name': property_name,
                       'property_source': property_source,
                       'property_type': property_type})
    df['pid'] = pid
    return df

In [4]:
properties = service.management().webproperties().list(accountId='66188758').execute()

In [5]:
df_all = pd.DataFrame()
for item in properties.get('items'):

    df_profile = read_profiles_per_property(service, item)
    df_all = df_all.append(df_profile, ignore_index=True)

In [7]:
df_all

,ua_profile_id,name,property_name,property_source,property_type,pid
0,106630793,"A - App, BC - Overview",3 - App - Brand - Bestseller.com,App,Brand,UA-66188758-1
1,106645875,"B - App, BC - United Kingdom",3 - App - Brand - Bestseller.com,App,Brand,UA-66188758-1
2,106647798,"B - App, BC - Switzerland",3 - App - Brand - Bestseller.com,App,Brand,UA-66188758-1
3,106649030,"B - App, BC - Denmark",3 - App - Brand - Bestseller.com,App,Brand,UA-66188758-1
4,106649048,"B - App, BC - Belgium",3 - App - Brand - Bestseller.com,App,Brand,UA-66188758-1
...,...,...,...,...,...,...
881,179325638,Web - Market - Netherlands (Non User-ID reports),Roll-Up Property,N/A,N/A,UA-66188758-54
882,179326120,Web - Market - Italy (Non User-ID reports),Roll-Up Property,N/A,N/A,UA-66188758-54
883,179332248,Web - Market - Spain (Non User-ID reports),Roll-Up Property,N/A,N/A,UA-66188758-54
884,179332487,Web - Market - Overview (Non User-ID reports),Roll-Up Property,N/A,N/A,UA-66188758-54


### Delete userid accounts

In [13]:
df_user_id = df_all.loc[(df_all.name.str.find('(User ID')>=0)&(df_all.name.str.find('Channa')<0)]
df_user_id.head()

,ua_profile_id,name,property_name,property_source,property_type,pid


In [14]:
# they are all deleted now
len(df_user_id)

0

In [8]:
l = list(zip(df_user_id.ua_profile_id,df_user_id.pid,df_user_id.name))

In [9]:
import time
for ua_id, pid, name in l:
    print(f'Deleting {name}')
    service.management().profiles().delete(accountId='66188758', webPropertyId=pid, profileId=ua_id).execute()
    time.sleep(0.1)

Deleting B - Web, GBR - Bianco.com (User ID)
Deleting B - Web, SLH - Austria (User ID)
Deleting B - Web, SLH - Sweden (User ID)
Deleting B - Web, SLH - United Kingdom (User ID)
Deleting B - Web, SLH - Denmark (User ID)
Deleting B - Web, SLH - Spain (User ID)
Deleting B - Web, SLH - Switzerland (User ID)
Deleting C - Web, SLH - Rest of World (User ID)
Deleting B - Web, SLH - Norway (User ID)
Deleting B - Web, SLH - Germany (User ID)
Deleting B - Web, SLH - France (User ID)
Deleting B - Web, SLH - Finland (User ID)
Deleting B - Web, SLH - Belgium (User ID)
Deleting B - Web, SLH - Netherlands (User ID)
Deleting B - Web, SLH - Italy (User ID)
Deleting B - Web, SLH - Ireland (User ID)
Deleting B - Web, BI - Norway (User ID)
Deleting B - Web, BI - France (User ID)
Deleting B - Web, BI - Germany (User ID)
Deleting B - Web, BI - Netherlands (User ID)
Deleting B - Web, BI - United Kingdom (User ID)
Deleting B - Web, BI - Sweden (User ID)
Deleting B - Web, BI - Austria (User ID)
Deleting B - Web

## Filters

### List all filters 

In [91]:
filters = service.management().filters().list(accountId='66188758').execute()

In [92]:
# a example of how filters are displayed
filters.get('items')[0]

{'id': '22488494',
 'kind': 'analytics#filter',
 'selfLink': 'https://www.googleapis.com/analytics/v3/management/accounts/66188758/filters/22488494',
 'accountId': '66188758',
 'name': 'CountryCode = NLD',
 'type': 'INCLUDE',
 'created': '2015-08-11T13:47:32.257Z',
 'updated': '2015-08-13T12:33:56.849Z',
 'parentLink': {'type': 'analytics#account',
  'href': 'https://www.googleapis.com/analytics/v3/management/accounts/66188758'},
 'includeDetails': {'kind': 'analytics#filterExpression',
  'field': 'CUSTOM_DIMENSION',
  'matchType': 'MATCHES',
  'expressionValue': 'NLD',
  'caseSensitive': False,
  'fieldIndex': 1}}

In [94]:
# convert the dic formatted info to DataFrame for readability.
id_ = [x.get('id') for x in filters.get('items')]
name_ = [x.get('name') for x in filters.get('items')]
type_ = [x.get('type') for x in filters.get('items')]

field_ = [x.get(x.get('type').lower()+'Details').get('field') for x in filters.get('items')]
matchtype_ = [x.get(x.get('type').lower()+'Details').get('matchType') for x in filters.get('items')]
expvalue_ = [x.get(x.get('type').lower()+'Details').get('expressionValue') for x in filters.get('items')]
casesensitive_ = [x.get(x.get('type').lower()+'Details').get('caseSensitive') for x in filters.get('items')]
fieldindex_ = [x.get(x.get('type').lower()+'Details').get('fieldIndex') for x in filters.get('items')]
df_filter = pd.DataFrame({'id':id_
                          ,'name':name_
                          ,'type':type_
                          ,'field':field_
                          ,'matchtype':matchtype_
                          ,'expvalue':expvalue_
                          ,'casesensitive':casesensitive_
                          ,'fileldindex':fieldindex_
                           })

In [95]:
# field index = 1 actually means cd.index = 1
df_filter

,id,name,type,field,matchtype,expvalue,casesensitive,fileldindex
0,22488494,CountryCode = NLD,INCLUDE,CUSTOM_DIMENSION,MATCHES,NLD,False,1.0
1,22526432,CountryCode = DEU,INCLUDE,CUSTOM_DIMENSION,MATCHES,DEU,False,1.0
2,22528432,CountryCode = BEL,INCLUDE,CUSTOM_DIMENSION,MATCHES,BEL,False,1.0
3,22528529,CountryCode = IRL,INCLUDE,CUSTOM_DIMENSION,MATCHES,IRL,False,1.0
4,22528819,CountryCode = SWE,INCLUDE,CUSTOM_DIMENSION,MATCHES,SWE,False,1.0
...,...,...,...,...,...,...,...,...
201,80755345,SiteCoutry=Denmark(new),INCLUDE,CUSTOM_DIMENSION,MATCHES,DK,False,1.0
202,80765258,SiteCoutry=Germany(new),INCLUDE,CUSTOM_DIMENSION,MATCHES,DE,False,1.0
203,80765364,Exclude Bestseller Traffic 83.151.148(new),EXCLUDE,GEO_IP_ADDRESS,MATCHES,^83\.151\.148\.([1-9]|[1-9][0-9]|1([0-9][0-9])...,False,NaN
204,80770373,Exclude Ikwilbovenaan(new),EXCLUDE,GEO_IP_ADDRESS,EQUAL,185.127.111.252,False,NaN


### Create Site Country Filters

There are 3 different kinds of filters (include, exclude, advanced), so I did them in 3 sections.

In [11]:
countries= [('Austria','AT'),
             ('Belgium','BE'),
             ('Denmark','DK'),
             ('Finland','FI'),
             ('France','FR'),
             ('Germany','DE'),
             ('Ireland','IE'),
             ('Italy','IT'),
             ('Netherlands','NL'),
             ('Norway','NO'),
             ('Poland','PL'),
             ('Spain','ES'),
             ('Sweden','SE'),
             ('Switzerland','CH'),
             ('United Kingdom','GB'),
             ('Rest of world','RW')]

In [17]:
# inserting filters for every country
for countryname, countrycode in countries:
    service.management().filters().insert(
          accountId='66188758',
          body={
              'name': f'SiteCoutry={countryname}(new)',
              'type': 'INCLUDE',
              'includeDetails': {
                  'field': 'CUSTOM_DIMENSION',
                  'matchType': 'MATCHES',
                  'expressionValue': countrycode,
                  'caseSensitive': False,
                  'fieldIndex': 1
                  }
          }
      ).execute()

### Create Exclude IP filters

In [29]:
# This are exsiting IP filters in GA
exc_l=['39344710','26455472','26111742','39351209','47393090','26133671','55936743']
df_exc = df_filter.loc[df_filter.id.isin(exc_l)]
df_exc

,id,name,type,field,matchtype,expvalue,casesensitive,fileldindex
109,26111742,Exclude Site Confidence Browser Bot,EXCLUDE,BROWSER,MATCHES,SiteCon Browser,True,NaN
111,26133671,Exclude Bestseller Traffic 83.151.148,EXCLUDE,GEO_IP_ADDRESS,MATCHES,^83\.151\.148\.([1-9]|[1-9][0-9]|1([0-9][0-9])...,False,NaN
128,26455472,Exclude QuantAds Traffic,EXCLUDE,GEO_IP_ADDRESS,MATCHES,X.X.X.X,False,NaN
146,39344710,Exclude Bestseller 80.197.218.183,EXCLUDE,GEO_IP_ADDRESS,MATCHES,80.197.218.183,False,NaN
147,39351209,Exclude Bestseller Traffic 185.9.141,EXCLUDE,GEO_IP_ADDRESS,MATCHES,^185\.9\.141\.([1-9]{1}|1[0-4]{1})$,False,NaN
165,47393090,Exclude Bestseller - Amsterdam,EXCLUDE,GEO_IP_ADDRESS,MATCHES,83.219.74.155,False,NaN
173,55936743,Exclude Ikwilbovenaan,EXCLUDE,GEO_IP_ADDRESS,EQUAL,185.127.111.252,False,NaN


In [38]:
# I used all the origin settings here to create our own new fitlers
# The only difference is I changed "Exclude Site Confidence Browser Bot" fitler to casesensitive = False
l = list(zip(df_exc.name,df_exc.type,df_exc.field,df_exc.matchtype,df_exc.expvalue,df_exc.casesensitive))
for name,type,field,matchtype,expvalue,casesensitive in l:
    service.management().filters().insert(
          accountId='66188758',
          body={
              'name': f'{name}(new)',
              'type': 'EXCLUDE',
              'excludeDetails': {
                  'field': field,
                  'matchType': matchtype,
                  'expressionValue': expvalue,
                  'caseSensitive': False
                  }
          }
      ).execute()

### Create Whole URL filter

In [88]:
# whole URL filter also copied from exsiting fitler 
service.management().filters().insert(
          accountId='66188758',
          body={
              'name': 'Whole URL(new)',
              'type': 'ADVANCED',
              'advancedDetails':{
                  'fieldA': 'PAGE_REQUEST_URI',
                  'extractA': '(.*)',
                  'fieldB': 'PAGE_HOSTNAME',
                  'extractB': '(.*)',
                  'outputToField': 'PAGE_REQUEST_URI',
                  'outputConstructor': '$B1$A1',
                  'fieldARequired': True,
                  'fieldBRequired': True,
                  'overrideOutputField': True,
                  'caseSensitive': False}
          }
      ).execute()

{'id': '80729795',
 'kind': 'analytics#filter',
 'selfLink': 'https://www.googleapis.com/analytics/v3/management/accounts/66188758/filters/80729795',
 'accountId': '66188758',
 'name': 'Whole URL(new)',
 'type': 'ADVANCED',
 'created': '2020-03-16T17:12:23.568Z',
 'updated': '2020-03-16T17:12:23.568Z',
 'parentLink': {'type': 'analytics#account',
  'href': 'https://www.googleapis.com/analytics/v3/management/accounts/66188758'},
 'advancedDetails': {'fieldA': 'PAGE_REQUEST_URI',
  'extractA': '(.*)',
  'fieldB': 'PAGE_HOSTNAME',
  'extractB': '(.*)',
  'outputToField': 'PAGE_REQUEST_URI',
  'outputConstructor': '$B1$A1',
  'fieldARequired': True,
  'fieldBRequired': True,
  'overrideOutputField': True,
  'caseSensitive': False}}

### Create new views and link to filters

In [97]:
# First find all the new filters we just created ('new' in filter name)
df_new_filters = df_filter.loc[df_filter.name.str.find('new')>0]
exclude_filter_ids = df_new_filters.loc[df_new_filters.type=='EXCLUDE','id'].values.tolist()
advanced_filter_ids = df_new_filters.loc[df_new_filters.type=='ADVANCED','id'].values.tolist()

In [100]:
propertyid = 'UA-66188758-51' # SL property as a test

In [101]:
# create country views for SL property and link them directly with our new fitlers
for countryname, countrycode in countries:

    # prepare all filter ids
    country_filter_id = df_new_filters.loc[df_new_filters.expvalue==countrycode,'id'].values.tolist()
    all_filters_ids = exclude_filter_ids + advanced_filter_ids+country_filter_id

    # create view
    new_view = service.management().profiles().insert(
          accountId='66188758',
          webPropertyId=propertyid,
          body={
              'name': f'Web - SL - {countryname}',
              'eCommerceTracking': True,
          }
      ).execute()

    new_view_id = new_view.get('id')

    # link filters to view
    for filter_id in all_filters_ids:
        service.management().profileFilterLinks().insert(
              accountId='66188758',
              webPropertyId=propertyid,
              profileId = new_view_id,
              body={'filterRef': {'id': filter_id}}
          ).execute()